# Youth On Record - JSON parsing

With the goal of creating a database to store data for Non-profit Youth On Record (See https://www.youthonrecord.org/), the first challenge is gathering data from disparate sources, and doing some ETL work to get them formatted the way we need before pushing to the new database.

The first data visualization project was to create an interactive workshop map going back to late 2019. The map, which has it's current home in Heroku, can be seen here: https://lit-ocean-77747.herokuapp.com/ . At the outset, the data needed for this map was from a variety of spreadsheets and google docs, and I decided to hard-code this directly into the React web application as the data had no home from which it could be retrieved to generate the map markers - not a best practice!

In this ETL project, we can now deconstruct the original dataset. We had three sets of map markers - the first for school partnerships, the second for libraries, and the third for onsite workshops conducted at the Youth Media Studio downtown. Each was originally stored as JSON in a javascript variable, and has been exported to JSON files here (see the json_data directory) which we can pull into pandas and begin parsing. Meanwhile, each file has nested objects indicating who the teacher was, what class was taught, and other info which we can split up to be moved to separate tables in the new PostgreSQL database.

In [2]:
# dependencies
import pandas as pd 
from pprint import pprint

In [5]:
# load the schools json
school_json_path = "./json_data/yor_schools.json"
schools_json_df = pd.read_json(school_json_path)
schools_json_df

,name,address,url,classes,teachers,latitude,longitude,img_src
0,Colorado High School Charter Osage,"1175 Osage St #100, Denver CO 80204",https://www.coloradohighschoolcharter.com/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Devin Urioste', 'teacher_li...",39.734798,-105.008013,
1,Colorado High School Charter GES,3093 E. 42nd Ave Denver CO 80216,https://www.coloradohighschoolcharter.com/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Elijah Lynch', 'teacher_lin...",39.754973,-105.013325,
2,North High School,"2960 Speer Blvd, Denver CO 80211",https://north.dpsk12.org/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Mona Magno', 'teacher_link'...",39.754962,-105.013497,
3,Legacy Options High School,"6850 Argonne St, Denver CO 80249",https://www.makeyourownlegacy.org/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Michelle Rocqet', 'teacher_...",39.820655,-104.771209,
4,Third Way Center,"505 W 5th Ave, Denver CO 80204",https://thirdwaycenter.org/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Babah Fly', 'teacher_link':...",39.734261,-104.970029,
5,Rise Up Community School,"2342 Broadway, Denver CO 80205",https://riseupcommunityschool.net/,[{'class_title': 'Introduction to Music Produc...,"[{'teacher_name': 'Michelle Rocqet', 'teacher_...",39.755475,-104.989392,
6,PREP Academy,"2727 Columbine St, Denver CO 80205",http://prepacademy.dpsk12.org/,"[{'class_title': 'Creative Writing', 'class_li...","[{'teacher_name': 'Elijah Lynch', 'teacher_lin...",39.755454,-105.057421,
7,Aurora West Collegiate Prep Academy,"10100 E 13th Ave, Aurora CO 80010",https://awcpa.aurorak12.org/,[{'class_title': 'Introduction to Ethnic Studi...,"[{'teacher_name': 'Stephen Brackett', 'teacher...",39.735981,-104.872784,
8,Hill Campus of Arts and Sciences,"451 Clermont Street, Denver CO 80220",http://hill.dpsk12.org/,"[{'class_title': 'Intro to Creative Writing', ...","[{'teacher_name': 'Babah Fly', 'teacher_link':...",39.723709,-104.937673,


In [7]:
#  load the libraries json
library_json_path = "./json_data/yor_libraries.json"
libraries_json_df = pd.read_json(library_json_path)
libraries_json_df

,name,address,url,classes,teachers,latitude,longitude,img_src
0,Ford Warren,"2825 N High St, Denver CO 80205",https://www.denverlibrary.org/content/ford-war...,"[{'class_title': 'Beats Lab', 'class_link': ''}]","[{'teacher_name': 'Mona Magno', 'teacher_link'...",39.757389,-104.967268,https://geo2.ggpht.com/cbk?panoid=_DAxCFBBb-bp...
1,Corky Gonzalez,"1498 Irving St, Denver CO 80204",https://www.denverlibrary.org/content/rodolfo-...,"[{'class_title': 'Beats Lab', 'class_link': ''}]","[{'teacher_name': 'Elijah Lynch', 'teacher_lin...",39.740052,-105.031748,https://geo0.ggpht.com/cbk?panoid=qb_c23eijRyK...
2,Ross Barnum,"3570 W First Ave, Denver CO 80219",https://www.denverlibrary.org/content/ross-bar...,"[{'class_title': 'Beats Lab', 'class_link': ''}]","[{'teacher_name': 'Michelle Rocqet', 'teacher_...",39.717953,-105.036795,https://geo2.ggpht.com/cbk?panoid=G_mBcqGpr6JM...
3,Athmar Park,"1055 S Tejon St, Denver CO 80223",https://www.denverlibrary.org/content/athmar-p...,"[{'class_title': 'Beats Lab', 'class_link': ''}]","[{'teacher_name': 'Devin Urisote', 'teacher_li...",39.697086,-105.015794,https://geo0.ggpht.com/cbk?panoid=m9snCv6AFMd2...
4,Pauline Robinson,"5575 E 33rd Ave, Denver CO 80207",https://www.denverlibrary.org/content/pauline-...,"[{'class_title': 'single day kick off', 'class...","[{'teacher_name': 'Stephen Brackett', 'teacher...",39.764098,-104.924881,https://geo1.ggpht.com/cbk?panoid=Q9dE1rc5JGYa...


In [8]:
# load the onsite workshops json
wkshop_json_path = "./json_data/yor_onsite_workshops.json"
workshop_json_df = pd.read_json(wkshop_json_path)
workshop_json_df

,partner,students_served,date
0,Heritage Camps for Adoptive Families,20,6/17/2019
1,EcoArts,10,3/21/2019
2,Noel Community Arts,15,12/14/2019
3,Bruce Randolph,10,4/30 & 5/1/2019
4,Kipp middle school,10,4/30/2019
5,World Denver,20,3/20/2019
6,Gateway to College,15,5/20/2019
7,RAMP,10,2/28/2019
8,John F Kennedy High School,6,5/14/2019
9,Florence Crittenton High School,15,5/16/2019


## ETL plan
- as we want to have a table to store all partnerships, we can vertically merge the schools and libraries with a pd.concat - these have a lot of common fields, and we can drop the classes and teachers columns
- we do want classes and teachers for use in a differen table, so we can join on their id to the school they taught
- the onsite workshops will be a bit of stick in the mud, since we did not bother collecting things like address or geographic coordinates for these as they were all conducted at the Youth Media Studio on 10th and Navajo downtown. Furthermore, many of these were a one day workshop, whereas others have a date range...and include a 'students_served' column that we do not have for the rest.

In [10]:
# concat the schools and libraries, drop teachers and classes...first let's add a 'type' to each of these
schools_json_df['type'] = "school"
# ^^^ how granular should we be? While there's not a ton, it might be better down the road to say 'middle school' v. 'high school' to be able to count students in various age demographics? anyway...
libraries_json_df['type'] = "library"

In [16]:
offsite_wkshop_df = pd.concat([schools_json_df, libraries_json_df])
# drop classes and teachers...and img_src while we're at it since those were copied and pasted from google for the map, and it's null for the schools anyway
offsite_wkshop_df = offsite_wkshop_df[['name', 'address', 'url', 'latitude', 'longitude','type']]
offsite_wkshop_df
# TODO - the index is wonky...reset_index() adds a new index but does not remove the old one?
# TODO - address should be split into street, city, state, zip
# TODO - type could be more granular?

,name,address,url,latitude,longitude,type
0,Colorado High School Charter Osage,"1175 Osage St #100, Denver CO 80204",https://www.coloradohighschoolcharter.com/,39.734798,-105.008013,school
1,Colorado High School Charter GES,3093 E. 42nd Ave Denver CO 80216,https://www.coloradohighschoolcharter.com/,39.754973,-105.013325,school
2,North High School,"2960 Speer Blvd, Denver CO 80211",https://north.dpsk12.org/,39.754962,-105.013497,school
3,Legacy Options High School,"6850 Argonne St, Denver CO 80249",https://www.makeyourownlegacy.org/,39.820655,-104.771209,school
4,Third Way Center,"505 W 5th Ave, Denver CO 80204",https://thirdwaycenter.org/,39.734261,-104.970029,school
5,Rise Up Community School,"2342 Broadway, Denver CO 80205",https://riseupcommunityschool.net/,39.755475,-104.989392,school
6,PREP Academy,"2727 Columbine St, Denver CO 80205",http://prepacademy.dpsk12.org/,39.755454,-105.057421,school
7,Aurora West Collegiate Prep Academy,"10100 E 13th Ave, Aurora CO 80010",https://awcpa.aurorak12.org/,39.735981,-104.872784,school
8,Hill Campus of Arts and Sciences,"451 Clermont Street, Denver CO 80220",http://hill.dpsk12.org/,39.723709,-104.937673,school
0,Ford Warren,"2825 N High St, Denver CO 80205",https://www.denverlibrary.org/content/ford-war...,39.757389,-104.967268,library
